#Prep

In [ ]:
!pip install osmnx
!pip install --upgrade xlrd

In [ ]:
import osmnx as ox
import matplotlib
import networkx as nx
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import shapely
import geopandas as gpd
import itertools

In [ ]:
url="https://docs.google.com/spreadsheets/d/1iASclM8Hwaro9TQEa2AnBs6fDo2_AQgp/edit?usp=sharing&ouid=108747149587154465757&rtpof=true&sd=true"
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

In [ ]:
geocodes_dataframe=pd.read_excel(url, engine = 'openpyxl')
geocodes_dataframe

,Geocode,Location
0,"Via Pietro Secondo Orseolo, Grado, Gorizia, Fr...","45.6772, 13.3863"
1,"Viale delle Terme, Lignano Pineta, Lignano Sab...","45.6636, 13.1045"
2,"Viale Aurora, Bibione, San Michele al Tagliame...","45.6354, 13.0483"
3,"Sottomarina, Chioggia, Venezia, Veneto, 30015,...","45.2026, 12.2928"
4,"Parco della Rimembranza, Cervia, Ravenna, Emil...","44.2606, 12.3517"
...,...,...
105,"Borgo Castello, Loano, Savona, Liguria, 17025,...","44.1257, 8.2515"
106,"Colla Micheri, Andora, Savona, Liguria, 17053,...","43.9547, 8.1464"
107,"Diano Calderina, Imperia, Liguria, 18013, Italy","43.9122, 8.08"
108,"Galleria Francia, Sanremo, Imperia, Liguria, 1...","43.8185, 7.7735"


In [ ]:
cities =[i for i in geocodes_dataframe.Location] 

cities =[tuple(map(float, i.split(', '))) for i in cities] 
type(cities)

list

#dataset preparation for GRAN

In [ ]:
edges_final = pd.DataFrame()
nodes_final = pd.DataFrame()

startind=0
indicator = 1

for city in cities[:41]:
  G = ox.graph_from_point(city, dist=800, network_type='drive', simplify=True) 
  #retrieving nodes and edges as geodataframe from the city graph
  nodes, edges = ox.utils_graph.graph_to_gdfs(G)

  #creating a dictionary for changing the nodes indicies
  nodes['ndID'] = range(startind, startind + len(nodes))
  node_ind = dict(zip(nodes.index, nodes.ndID))

  startind+= len(nodes)

  #creating the relational long and lat
  xmin = nodes.x.min()
  ymin = nodes.y.min()

  nodes['xup'] = nodes.x -xmin
  nodes['yup'] = nodes.y -ymin

  #graph indicator for every node
  nodes['indicator']= indicator
  indicator+=1

  #updating the node indicies in edges
  edges1 = edges.copy()

  edges['ndstart']=edges.index.get_level_values(0)
  edges['ndend']=edges.index.get_level_values(1)

  edges["ndstart"].replace(node_ind, inplace=True)
  edges["ndend"].replace(node_ind, inplace=True)

  edges1['ndstart']=edges.index.get_level_values(1)
  edges1['ndend']=edges.index.get_level_values(0)

  edges1["ndstart"].replace(node_ind, inplace=True)
  edges1["ndend"].replace(node_ind, inplace=True)



  edges_final= edges_final.append(edges)
  edges_final= edges_final.append(edges1)
  nodes_final= nodes_final.append(nodes)

Cities_A  = pd.DataFrame().assign(ndstart=edges_final['ndstart'], ndend=edges_final['ndend'])
Cities_graph_indicator = pd.DataFrame().assign(indicator=nodes_final['indicator'])
Cities_node_labels = pd.DataFrame().assign(x=nodes_final['xup'], y=nodes_final['yup'])

Cities_A = Cities_A.reset_index(drop=True)
Cities_graph_indicator = Cities_graph_indicator.reset_index(drop=True)
Cities_node_labels = Cities_node_labels.reset_index(drop=True)

In [ ]:
a = 0
for city in cities[:41]:
  G = ox.graph_from_point(city, dist=800, network_type='drive', simplify=True) 
  #retrieving nodes and edges as geodataframe from the city graph
  nodes, edges = ox.utils_graph.graph_to_gdfs(G)
  if (len(nodes))>a:a = len(nodes)

In [ ]:
a

756

In [ ]:
#saving text files
np.savetxt(r'data/Cities/Cities_A.txt', Cities_A.values, delimiter= ",", fmt='%d' )
np.savetxt(r'data/Cities/Cities_graph_indicator.txt', Cities_graph_indicator.values, fmt='%d' )
np.savetxt(r'data/Cities/Cities_node_labels.txt', Cities_node_labels.values, delimiter= ",", fmt='%f')